In [1]:
import pandas as pd
from transformers import pipeline

2024-03-23 15:35:10.454000: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-23 15:35:10.612264: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 15:35:10.612326: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 15:35:10.644964: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-23 15:35:10.693508: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-23 15:35:10.695024: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
sgs = pd.read_csv('../working-csvs/fomc_sents.csv', index_col=0)

In [3]:
sgs

,lname,date,section,content
0,barkin,2018-01-31,1,"Thank you, Madam Chair."
1,barkin,2018-01-31,1,I am privileged to be here to participate in t...
2,barkin,2018-01-31,1,So thank you for that.
3,barkin,2018-01-31,1,"Speaking of humility, I am humbled to be here ..."
4,barkin,2018-01-31,1,I hope at the end of my remarks you won't thin...
...,...,...,...,...
261983,yellen,2018-01-31,2,Further comments?
261984,yellen,2018-01-31,2,Okay.
261985,yellen,2018-01-31,2,"Well, hearing this discussion, I think my incl..."
261986,yellen,2018-01-31,2,I recognize that there is some risk in doing i...


In [4]:
pipe = pipeline("text-classification", model="ProsusAI/finbert")

In [5]:
sgs['sentiment'] = pipe(sgs['content'].tolist())

runtime: 189m 7.2s

In [19]:
sgs['sent_category'] = sgs['sentiment'].map(lambda x : x['label'])
sgs['sent_prob'] = sgs['sentiment'].map(lambda x : x['score'])

In [20]:
def sent_map(category):
    if category == 'neutral':
        return 0
    elif category == 'positive':
        return 1
    return -1

In [24]:
sgs['sent'] = sgs['sent_category'].map(sent_map)

In [34]:
sgs.drop(columns=['sentiment', 'sent_category'], inplace=True)

In [36]:
sgs.to_csv('../working-csvs/fomc-sents-w-sentiment.csv')